<a href="https://colab.research.google.com/github/9645258/aiffel/blob/main/02_goingdeeper/01_nlp/%5BGD-08%5Dnlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 00

In [ ]:
!sudo apt-get install g++ openjdk-8-jdk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
g++ is already the newest version (4:7.4.0-1ubuntu2.3).
openjdk-8-jdk is already the newest version (8u312-b07-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 71 not upgraded.


In [ ]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **GD NLP 08 : Seq2seq으로 번역기 만들기**

- **한영 번역기 만들기**  

- **번역기 모델 학습에 필요한 텍스트 데이터 전처리가 한국어 포함하여 잘 이루어졌다.**  
구두점, 대소문자, 띄어쓰기, 한글 형태소분석 등 번역기 모델에 요구되는 전처리가 정상적으로 진행되었다.  

- **Attentional Seq2seq 모델이 정상적으로 구동된다.**  
seq2seq 모델 훈련 과정에서 training loss가 안정적으로 떨어지면서 학습이 진행됨이 확인되었다.  

- **테스트 결과 의미가 통하는 수준의 번역문이 생성되었다.**  
테스트용 디코더 모델이 정상적으로 만들어져서, 정답과 어느 정도 유사한 영어 번역이 진행됨을 확인하였다.  

## **1. 데이터 준비**


### **1-1. 라이브러리 import**

In [ ]:
import re
import os
import io
import random
import tarfile
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from konlpy.tag import Mecab

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.font_manager as fm

### **1-2. 데이터 import**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- **폰트 경로 설정**

In [ ]:
fontpath = '/content/drive/MyDrive/Colab Notebooks/aiffel/01_data/ROKAF Sans Bold.ttf'
font = fm.FontProperties(fname = fontpath)
plt.rc('font', family='ROKAF Sans')
mpl.font_manager.findfont(font)
%config InlineBackend.figure_format = 'retina'

- **국문 및 영문 train 데이터 import**

In [ ]:
path_to_en = "/content/drive/MyDrive/Colab Notebooks/aiffel/01_data/02-1_nlp/08/korean-english-news-v1/train/korean-english-park.train.en"
path_to_ko = "/content/drive/MyDrive/Colab Notebooks/aiffel/01_data/02-1_nlp/08/korean-english-news-v1/train/korean-english-park.train.ko"

In [ ]:
with open(path_to_ko, "r") as f:
    kor_raw = f.read().splitlines()

In [ ]:
with open(path_to_en, "r") as f:
    eng_raw = f.read().splitlines()

### **1-3. 데이터 확인**

- **데이터 수 확인**

In [ ]:
print('국문 train 데이터 수:', len(kor_raw))
print('영문 train 데이터 수:', len(eng_raw))

국문 train 데이터 수: 94123
영문 train 데이터 수: 94123


- **데이터 확인**

In [ ]:
print('국문 train 데이터:\n', random.sample(kor_raw, 30))
print('\n')
print('영문 train 데이터:\n', random.sample(eng_raw, 30))

국문 train 데이터:
 ['중국이 2008년 베이징 올림픽 성화봉송과 관련, 성화 봉송로에 대만을 포함시키며 “중국의 다리”라는 표현을 써 대만과 정치적인 마찰을 빚고 있다.', 'initially 처음에, 첫머리에', "홍콩 영화 '무간도'를 미국판으로 리메이크한 '사자(死子)들'은 레오나르도 디카프리오가 보스톤 범죄 조직 그늘 밑의 경찰로, 맷 데이먼이 야비한 갱으로, 잭 니콜슨이 모든 이들을 배후 조종하는 무자비한 갱 두목으로 등장한다.", '중국의 가족계획정책이 기혼자들의 1가구 1자녀 정책을 무시하면서 “인구 재팽창” 국변을 맞고 있다.', '베이징 공항에서 중국 여행객들은 힐 대표의 사진을 찍으려고 몰려들기도 한다고 이름을 밝히지 않은 힐 대표의 한 경호원이 말했다.', '그는“미움이나 악의로 영화를 제작할 수 없고 아무런 악감정도 이 영화에 반영되지 않았다”며 “부시의 정치를 싫어하지만 이상하고 엉뚱한 어색한 방법으로 마음을 끌어당겨 미국의 마음을 사로 잡았던 부시가 영화배우 존 웨인과 더 닮았다는 것을 알고 있다”고 말했다.', '농민들의 파업이 이어지며 크리스티나 페르난데스 아르헨티나 대통령 정부는 난관에 봉착했다.', '시라크는 "이란이 어느 곳에 핵무기를 떨어뜨릴 것인가.', '최근 조지 부시 미국 대통령이 이스라엘과 팔레스타인을 순방했으며 그의 순방일정 3일째, 중동평화회담과 관련한 이 같은 발표가 있었다.', '지난해 6월 이후 아기 엄마가 된 데이븐포트(31)는 개인적인 사정으로 대회에 참여하지 못한다고 밝혔다.', '이번 사고는 MBC 지역 방송국에서 주최한 콘서트를 보기 위해 5천여 명의 시민이 기다리고 있던 운동장 입구에서 벌어졌다.', '영국 석간 이브닝 스탠더드는 매들린과 얼굴이 비슷한 사진의 소녀는 인근 올리브 농장주의 딸이라고 설명했다.', '박태환은 한국인으로는 처음 올림픽을 제외한 국제 수영 대회에서 메달을 땄다.', '또 10명 중 1명은 계속된 휴대폰 사용으로 다른 사람들로부터 비난을 받을 적이 있다고 말했다.'

### **1-4. 데이터 병합**

- **국문/영문 데이터 수가 같으므로 한개의 데이터셋으로 변환**

In [ ]:
dataset = pd.DataFrame(zip(kor_raw, eng_raw), columns = ["kor", "eng"])

- **데이터 확인**

In [ ]:
dataset.sample(10)

,kor,eng
72362,이는 체코 출신 예술가 데이비드 서니가 만든 논란의 여지가 많은 설치 예술작품에 따...,That's the view of the European Union accordin...
92523,"Biden, Palin Go Toe to Toe on the Issues 용호상박의...",The head of the FDIC says the agency is standi...
48870,하지만 6자회담은 2년여간 교착 상태에 빠졌으며 북한은 지난 10월 9일 핵실험을 ...,"But the talks have been stalled for two years,..."
92608,“ 3월 말이라는 점을 감안할 때 꽤 많은 양의 눈입니다.,"“ Pretty heavy snow, especially considering it..."
41323,유니버셜 익스프레스는 이번 경매 물품을 배카로로부터 500만 달러(약 50억원)에 ...,Universal bought the items from Vaccaro for $5...
18596,지난해부터 계속된 호우와 지난 몇 주간 볼리비아를 강타한 폭풍으로 볼리비아 전역에서...,"LA PAZ, Bolivia (CNN) President Evo Morales de..."
49272,노왁의 변호사인 도널드 리케박은 판사가 살인 미수혐의에 대해 증명할 수 있는 요인을...,"Her attorney, Donald Lykkebak, argued the judg..."
64021,[동영상]추신수 스리런 홈런 작렬!,Korea to double African aid by ...
89597,원문 & 독해설명 인쇄,The protesters later joined the anti-impeachme...
79324,Majority expect Roh’s proposal to be thwarted ...,A report released by the Information Ministry ...


### **1-5. 중복 데이터 및 결측치 확인**

- **데이터 정보 확인**  
결측치 없음  

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94123 entries, 0 to 94122
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   kor     94123 non-null  object
 1   eng     94123 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


- **중복 데이터 확인 및 삭제**

In [ ]:
print("train 데이터 수:", len(dataset))

dup_data = dataset.duplicated()
dup_data = dup_data.value_counts()
print("중복 train 데이터 수:", dup_data[True])

dataset.drop_duplicates(inplace = True)
print("최종 train 데이터 수:", len(dataset))

train 데이터 수: 94123
중복 train 데이터 수: 15155
최종 train 데이터 수: 78968


### **1-6. 데이터 문장 길이 확인**

In [ ]:
def show_sentence_length(sentence_num, title, range_=[0, 500]):
    plt.figure(figsize=(13, 5))
    plt.suptitle(title, fontsize=14)
    
    plt.subplot(1, 2, 1)
    plt.hist(sentence_num, bins=range_[1], range=range_, facecolor='b', label='train')
    plt.xlabel('Number of question')
    plt.ylabel('Count of question')
    
    plt.subplot(1, 2, 2)
    plt.boxplot(sentence_num, labels=['token counts'], showmeans=True)

    plt.show()
    
    print("< Sentence Info >".center(100, "="))
    print(f"길이 최대:    {np.max(sentence_num):4d}")
    print(f"길이 최소:    {np.min(sentence_num):4d}")
    print(f"길이 평균:    {np.mean(sentence_num):7.3f}")
    print(f"길이 표준편차: {np.std(sentence_num):7.3f}", end="\n\n")
    
    percentile25 = np.percentile(sentence_num, 25)
    percentile50 = np.percentile(sentence_num, 50)
    percentile75 = np.percentile(sentence_num, 75)
    percentileIQR = percentile75 - percentile25
    percentileMAX = percentile75 + percentileIQR * 1.5
    
    print(f" 25/100분위:  {percentile25:7.3f}")
    print(f" 50/100분위:  {percentile50:7.3f}")
    print(f" 75/100분위:  {percentile75:7.3f}")
    print(f" MAX/100분위: {percentileMAX:7.3f}")
    print(f" IQR: {percentileIQR:7.3f}")
    print("=" * 100)